<h1> NOTEBOOK TO CREATE 3D DLC PROJECT and CALIBRATE DEEPLAB CUT </h1>

Make sure you are running this notebook in properly setup conda enviroment. The nbkGPU enviroment set up on server 2 works properly. Note that while you can calibrate the 3D enviroment in isolation, using it to analyze video data require two pre-trained networks on data from the two cameras in the exact same position as the calibration footage.

In [1]:
#Errors here mean there are issues with the enviroment dependencies or PATH configuration
import numpy as np
import deeplabcut
import moviepy
import cv2
from moviepy.editor import *
import os
import math
task = '3D_Box1'
experimenter = 'spencer_loggia'

In [2]:
import tensorflow as tf

In [3]:
#allow video memory growth as network expands to avoid convolutional network errors
TF_FORCE_GPU_ALLOW_GROWTH = True
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [4]:
#let's make sure we see a GPU:
#tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4643214641430092034,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4772067494288617744
 physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:41:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17486555811201490204
 physical_device_desc: "device: 1, name: GeForce RTX 2080, pci bus id: 0000:81:00.0, compute capability: 7.5"]

Create new 3d tracking project (creates a new project directory with a unique config.yaml)

In [6]:
path = deeplabcut.create_new_project_3d(task, experimenter, num_cameras=2, working_directory='F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\')
path = path #r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\3D_calibratations\3D_06152020-spencer_loggia-2020-06-18-3d\config.yaml'

Created "F:\MysoreData\nbk\mouseVideoAnalysis\Box1\3D_Box1-spencer_loggia-2021-02-14-3d\camera_matrix"
Created "F:\MysoreData\nbk\mouseVideoAnalysis\Box1\3D_Box1-spencer_loggia-2021-02-14-3d\calibration_images"
Created "F:\MysoreData\nbk\mouseVideoAnalysis\Box1\3D_Box1-spencer_loggia-2021-02-14-3d\undistortion"
Created "F:\MysoreData\nbk\mouseVideoAnalysis\Box1\3D_Box1-spencer_loggia-2021-02-14-3d\corners"
Generated "F:\MysoreData\nbk\mouseVideoAnalysis\Box1\3D_Box1-spencer_loggia-2021-02-14-3d\config.yaml"

A new project with name 3D_Box1-spencer_loggia-2021-02-14-3d is created at F:\MysoreData\nbk\mouseVideoAnalysis\Box1 and a configurable file (config.yaml) is stored there. If you have not calibrated the cameras, then use the function 'calibrate_camera' to start calibrating the camera otherwise use the function ``triangulate`` to triangulate the dataframe


<h2> A quick note about calibration data </h2>
<p> do not rotate the checker board if possible. Make sure you do move it around the working space though. The checker board should be at least 8X6 squares. take at least 30 calibration images, though you may need up to 80. The next cell takes the two directories where images are stored and pairs, renames, and moves the data.

In [ ]:
data1 = "..\\calibrations_data\\cam1\\" + DATE + "\\" #folder with cam1 image set
data2 = "..\\calibrations_data\\cam2\\" + DATE + "\\"#folder with cam2 image set 

from shutil import copyfile

list1 = os.listdir(data1)
list2 = os.listdir(data2)
images = []

if len(list1) != len(list2):
    print ("List dimmensions mis-matched. Make sure the images are aligned in proper pairs! Trying to fix automatically...")
    print(len(list1))
    print(len(list2))
    
    if len(list1) > len(list2):
        for i in range(len(list1)):
            if int(list1[i][-6:-4]) > len(list2):
                list1.pop(i)
                break
    elif len(list2) > len(list1):
        for i in range(len(list2)):
            if int(list2[i][-6:-4]) > len(list1):
                list2.pop(i)
                break
                
if len(list1) != len(list2):
    print ("List dimmensions mis-matched. unable to repair")
    print(len(list1))
    print(len(list2))
else:
    images.append(list1)
    images.append(list2)

    Images = np.array(images)
    Images = np.transpose(Images)
    print(Images)
    print(path)
    for i in Images:
        if (i[0][0:8] != 'camera-1') | (i[1][0:8] != 'camera-2'):
            print (i[0][0:6])
            print ('bad image names')
        else:
            copyfile(data1 + i[0], path[:-11] + '\\calibration_images\\' + i[0])
            copyfile(data2 + i[1], path[:-11] + '\\calibration_images\\' + i[1])
    


Calibrate (remove the bad images):

In [ ]:
print(path)
deeplabcut.calibrate_cameras(path, cbrow = 8,cbcol = 8, calibrate=False, alpha=0.2)


calibrate for reals

In [ ]:
path = path
deeplabcut.calibrate_cameras(path, cbrow = 8,cbcol = 8, calibrate=True, alpha=0.9)

Check for undistortion in images (i.e. verify proper alignment). After this runs, CHECK THE SAVED IMAGES in the undistortion directory

In [ ]:
deeplabcut.check_undistortion(path, cbrow = 8,cbcol = 8)

At this point make sure that you have trained your both cameras on the training data. Then come back to do some nice 3d triangulation!

In [ ]:
video_for_tri = 'F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\tmp_combine_videos\\07112020_all_test'
video = os.listdir(video_for_tri)
for i in range(0,len(video)):
    subclip = video_for_tri + '\\' + video[i]
    outclip = video_for_tri + '\\' + 'C' + video[i][0:len(video[i])-3] + 'avi'
    print(outclip)
    clip = VideoFileClip(subclip).subclip(5, 10)
    clip.write_videofile(outclip, codec='mpeg4')


In [ ]:
video_for_tri = 'F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\tmp_combine_videos\\07112020_all_test'
path = path
deeplabcut.triangulate(path, video_for_tri, filterpredictions=True)

In [ ]:
path = path #r'F:\MysoreData\nbk\TestDLC\3DTest\3Dtest-spencer_loggia-2019-10-27-3d\config.yaml'

deeplabcut.create_labeled_video_3d(path, ['F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\tmp_combine_videos\\07112020_all_test'], videofolder='F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\tmp_combine_videos\\07112020_all_test')
